In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-2cd2  GCE       4                                       RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from builtins import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan  7 16:12 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

# configs

In [ ]:
# TODO: Put your bucket name below and make sure you can access it without an error
bucket_name = 'fadlonbucket'

client = storage.Client()
blobs = client.list_blobs(bucket_name, prefix='postings_gcp')
# # print all the blobs in the bucket
# for b in blobs:
#     print(b.name)

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]  # TODO: calculate the corups stop words
all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stemmer = PorterStemmer()

def tokenize(text, use_stemmer=False):
  """
    This function aims in tokenize a text into a list of tokens.
    Moreover:
    * filter stopwords.
    * change all to lowwer case.
    * if use_stemmer = true -> use stemmer
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
  clean_text = []

  text = text.lower()
  tokens = [token.group() for token in RE_WORD.finditer(text)]
  for token in tokens:
    if token not in all_stopwords:
      if use_stemmer:
         token = stemmer.stem(token)
      clean_text.append(token)
  return clean_text

In [ ]:
def open_gcp(file_name):
  """
    get a read bit stream to a gcp blob in the bucket.
    start with the prefix: 'postings_gcp/'
    
    Parameters:
    -----------
    file_name: string , a path to the file.    
    
    Returns:
    -----------
    stream to read from
    """
    client = storage.Client(file_name)
    bucket = client.bucket(bucket_name)
    blob = bucket.get_blob('postings_gcp/' + file_name)
    return blob.open('rb')

def read_pickle(file_name):
  """
    read a pickle file from the bucket.
    from 'postings_gcp/'

    Parameters:
    -----------
    file_name: string , name of the file.    
    
    Returns:
    -----------
    object that readed from pickle
    """
    stream = open_gcp(file_name+".pkl")
    pick = pickle.load(stream)
    stream.close()
    return pick

# Building an Inverted Index

In [ ]:
#path to the full wiki data
full_path = "gs://wikidata_preprocessed/*"
#load to spark
parquetFile = spark.read.parquet(full_path)
# Count number of wiki pages
parquetFile.count()

6348910

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex, MultiFileReader

## read and write - posting list

In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings, name):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket_id` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
    name: string
      name of the index

  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  save_to = postings.map(lambda x: (token2bucket_id(x[0]), x))
  save_to = save_to.groupByKey().mapValues(list)
  return save_to.map(lambda x: InvertedIndex.write_a_posting_list(x, bucket_name, name))
    

TUPLE_SIZE = 6       
TF_MASK = 2 ** 16 - 1 # Masking the 16 low bits of an integer
from contextlib import closing

def read_posting_list(inverted, w):
  ''' read a posting list of a term from a inverted index. 

  Parameters:
  -----------
    inverted: Inverted_Index
      Inverted Index (w, posting_list) pair.
    w: string
      term to search

  Returns:
  --------
    List
      list of tuples (doc_id, frequency).
  '''
  with closing(MultiFileReader(bucket_name)) as reader:
    locs = inverted.posting_locs[w]
    b = reader.read(locs, inverted.df[w] * TUPLE_SIZE)
    posting_list = []
    for i in range(inverted.df[w]):
      doc_id = int.from_bytes(b[i*TUPLE_SIZE:i*TUPLE_SIZE+4], 'big')
      tf = int.from_bytes(b[i*TUPLE_SIZE+4:(i+1)*TUPLE_SIZE], 'big')
      posting_list.append((doc_id, tf))
    return posting_list

# Build Inverted Index:

In [ ]:
def word_count(tokens, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    tokens: list
      list of tokens
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  dictionary = {}
  for token in tokens:
    if token not in dictionary:
      dictionary[token] = [id, 1]
    else:
      dictionary[token][1] += 1
  return [(doc_id, (dictionary[doc_id][0], dictionary[doc_id][1])) for doc_id in dictionary]

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl, key=lambda x: x[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda x: (x[0], len(x[1])))

def f_sum(x):
  c=0
  for i in x:
    c += i
  return c
  
def calculate_term_total(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.mapValues(lambda x: [y[1] for y in x]).mapValues(f_sum)

build index:

In [ ]:
def build_inverted_index(key_value_pairs, name):
  '''
  building InvertedIndex and save it to the bucket in the clude.
  the data will be saved in 'postings_gcp/{name}_index/'

  Parameters:
  -----------
    key_value_pairs: RDD
      An RDD where each element is a (text, doc_id) pair.
    name: string
      name of the InvertedIndex, the data will be saved in 'postings_gcp/{name}_index/'

  Returns:
  --------
    Inverted_Index
     the new inverted index.
  '''
  word_counts = key_value_pairs.flatMap(lambda x: word_count(tokenize(x[0]), x[1]))
  postings = word_counts.groupByKey().mapValues(reduce_word_counts)
  postings = postings.filter(lambda x: len(x[1])>50)
  w2df = calculate_df(postings)
  w2df_dict = w2df.collectAsMap()
  _ = partition_postings_and_write(postings, name).collect() #TODO: _ was "posting_locs_list" before

  # collect all posting lists locations into one super-set
  super_posting_locs = defaultdict(list)
  for blob in client.list_blobs(bucket_name, prefix=f'postings_gcp/{name}_index/'):
    if not blob.name.endswith("pickle"):
      continue
    with blob.open("rb") as f:
      posting_locs = pickle.load(f)
      for k, v in posting_locs.items():
        super_posting_locs[k].extend(v)
  
  # build DL
  DL = key_value_pairs.map(lambda x: (x[1], len(tokenize(x[0]))))
  #build term_total
  term_total = calculate_term_total(postings).groupByKey().mapValues(f_sum)

  # Create inverted index instance
  index = InvertedIndex()
  # Adding the posting locations dictionary to the inverted index
  index.posting_locs = super_posting_locs
  # Add the token - df dictionary to the inverted index
  index.df = w2df_dict
  # Add the DL
  index.DL = dict(DL.collect())
  # term_total
  index.term_total = dict(term_total.collect())
  # write the global stats out
  index.write_index('.', name)

  # upload to gs
  index_src = name + ".pkl"
  index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
  !gsutil cp $index_src $index_dst
  !gsutil ls -lh $index_dst

  return index

In [ ]:
# rdd of text and doc_id
doc_text_pairs = parquetFile.select("text", "id").rdd
# rdd of title and doc_id
doc_title_pairs = parquetFile.select("title", "id").rdd
# rdd of anchor_text and doc_id
anchor_data = parquetFile.select("anchor_text").rdd

In [ ]:
#build body
body_index_name = "text"
inverted_body = build_inverted_index(doc_text_pairs, bodi_index_name)

#build title
title_index_name = "title"
inverted_title = build_inverted_index(doc_title_pairs, title_index_name)

#build anchor
def to_pairs(row):
    pairs = []
    for doc_id, anchor_text in row[0]:
        pairs.append((doc_id, anchor_text))
    return pairs

all_anchor_text = anchor_data.flatMap(to_pairs).groupByKey().mapValues(list).map(lambda x: (" ".join(x[1]), x[0]))
anchor_index_name = "anchor"
inverted_anchor = build_inverted_index(all_anchor_text, anchor_index_name)

In [ ]:
print("DONE!!!")